In [1]:

# Set up notebook
%pprint
import sys
if (osp.join(os.pardir, 'py') not in sys.path): sys.path.insert(1, osp.join(os.pardir, 'py'))

Pretty printing has been turned OFF


In [3]:

# load libraries
from FRVRS import (nu, fu, DataFrame, to_datetime, Series, np, osp, nan)
from datetime import date, timedelta
import os
from IPython.display import HTML
import pandas as pd

nu.delete_ipynb_checkpoint_folders()


# Get Injury Stats

In [4]:

# Get the standard data frames
data_frames_dict = nu.load_data_frames(
    verbose=False, first_responder_master_registry_df='', first_responder_master_registry_file_stats_df='',
    first_responder_master_registry_scene_stats_df=''
)
triage_paper_df = data_frames_dict['first_responder_master_registry_df']
print(triage_paper_df.shape)
file_stats_df = data_frames_dict['first_responder_master_registry_file_stats_df']
print(file_stats_df.shape)
scene_stats_df = data_frames_dict['first_responder_master_registry_scene_stats_df']
print(scene_stats_df.shape)
elevens_df = fu.get_elevens_dataframe(
    triage_paper_df, file_stats_df, scene_stats_df, needed_columns=[
        'scene_type', 'is_scene_aborted', 'is_a_one_triage_file', 'responder_category'
    ]
)
print(elevens_df.shape)

(401177, 111)
(332, 19)
(486, 48)
(399048, 115)


In [5]:

rows_list = []
for (session_uuid, scene_id, patient_id), patient_df in elevens_df.groupby(fu.patient_groupby_columns):
    for injury_id, injury_df in patient_df.groupby('injury_id'):
        row_dict = {cn: eval(cn) for cn in fu.injury_groupby_columns}
        row_dict['is_injury_correctly_treated'] = fu.get_is_injury_correctly_treated(injury_df)
        row_dict['is_injury_hemorrhage'] = fu.get_is_injury_hemorrhage(injury_df)
        row_dict['is_hemorrhage_tool_applied'] = fu.get_is_hemorrhage_tool_applied(injury_df, patient_df)
        row_dict['is_injury_severe'] = fu.get_is_injury_severe(injury_df)
        row_dict['is_bleeding_correctly_treated'] = fu.get_is_bleeding_correctly_treated(injury_df)
        row_dict['injury_correctly_treated_time'] = fu.get_injury_correctly_treated_time(injury_df)
        rows_list.append(row_dict)
df = DataFrame(rows_list)

In [6]:

columns_list = ['is_injury_correctly_treated', 'is_injury_hemorrhage', 'is_hemorrhage_tool_applied']
df2 = df.drop_duplicates(subset=columns_list).T
df2

,0,2,8,12
session_uuid,009b848c-ea64-4f22-bd40-711213a3d327,009b848c-ea64-4f22-bd40-711213a3d327,009b848c-ea64-4f22-bd40-711213a3d327,009b848c-ea64-4f22-bd40-711213a3d327
scene_id,0,0,0,0
patient_id,Bob_0 Root,Bob_0 Root,Gary_3 Root,Lily_2 Root
injury_id,Face Shrapnel,L Stomach Puncture,R Chest Collapse,R Shin Amputation
is_injury_correctly_treated,False,False,True,True
is_injury_hemorrhage,False,True,False,True
is_hemorrhage_tool_applied,False,False,False,False
is_injury_severe,False,True,False,True
is_bleeding_correctly_treated,False,False,False,True
injury_correctly_treated_time,NaN,NaN,941653.0,413178.0


In [11]:

mask_series = elevens_df.session_uuid.isin(df2.T.session_uuid) & elevens_df.scene_id.isin(df2.T.scene_id)
mask_series &= elevens_df.patient_id.isin(df2.T.patient_id) & elevens_df.injury_id.isin(df2.T.injury_id)

(7, 115)


In [32]:

from random import choices

scene_df = choices(scene_dfs_list, k=1)[0]
df = scene_df.sample(15).sort_values('action_tick')
mask_series =  df.injury_required_procedure.isin(fu.hemorrhage_control_procedures_list)
while not mask_series.sum() >= 5:
    df = scene_df.sample(15).sort_values('action_tick')
    mask_series =  df.injury_required_procedure.isin(fu.hemorrhage_control_procedures_list)
# mask_series = ~scene_df.injury_treated_injury_treated_with_wrong_treatment.isnull()
# df = scene_df[mask_series]
columns_list = [
    'action_tick', 'patient_id', 'injury_required_procedure', 'action_type', 'injury_id',
    'injury_record_required_procedure', 'injury_treated_required_procedure'
]
for cn in columns_list: print(f"""            '{cn}': {df[cn].tolist()},""")


    def setUp(self):
        # Create a sample DataFrame
        self.scene_df = DataFrame({
            'action_tick': [223819, 223819, 223819, 241120, 256019, 256924, 285654, 289814, 317108, 319906, 321245, 367706, 368149, 568501, 571875],
            'patient_id': ['Lily_4 Root', 'Lily_2 Root', 'Bob_0 Root', 'Gloria_6 Root', nan, nan, 'Lily_2 Root', nan, nan, nan, 'Mike_7 Root', nan, nan, 'Gloria_8 Root', nan],
            'injury_required_procedure': ['woundpack', 'tourniquet', 'tourniquet', nan, nan, nan, 'tourniquet', nan, nan, nan, 'tourniquet', nan, nan, nan, nan],
            'action_type': ['INJURY_RECORD', 'INJURY_RECORD', 'INJURY_RECORD', 'S_A_L_T_WAVE_IF_CAN', 'TOOL_HOVER', 'TOOL_HOVER', 'INJURY_TREATED', 'VOICE_CAPTURE', 'TOOL_HOVER', 'TOOL_HOVER', 'INJURY_TREATED', 'TOOL_HOVER', 'TOOL_HOVER', 'S_A_L_T_WALK_IF_CAN', 'TELEPORT'],
            'injury_id': ['L Side Puncture', 'R Shin Amputation', 'L Thigh Laceration', nan, nan, nan, 'R Shin Amputation', nan, nan, nan, 'L Th

In [10]:

df.is_hemorrhage_tool_applied.value_counts()

False    5932
Name: is_hemorrhage_tool_applied, dtype: int64

In [12]:

class_name = ''.join([w.title() for w in 'get_is_hemorrhage_tool_applied'.split('_')])
print(f"class Test{class_name}(unittest.TestCase)")

class TestGetIsHemorrhageToolApplied(unittest.TestCase)
